<a href="https://colab.research.google.com/github/caveligh/Colab_Projects/blob/main/WebscrapingDatabases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install mechanicalsoup
import mechanicalsoup
import pandas as pd
import sqlite3


     |████████████████████████████████| 97 kB 3.5 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
# create browser object & open URL
browser = mechanicalsoup.StatefulBrowser()
browser.open("https://en.wikipedia.org/wiki/Comparison_of_Linux_distributions")

<Response [200]>

In [ ]:
# extract all table headers (entire "Distribution" column)
th = browser.page.find_all("th", attrs={"class": "table-rh"})
# tidy up and slice off non-table elements 
distribution = [value.text.replace("\n", "") for value in th]
distribution = distribution[:95]

In [ ]:
distribution

['AlmaLinux',
 'Alpine Linux',
 'ALT Linux',
 'antiX',
 'ArchBang',
 'Arch Linux',
 'BLAG',
 'Bodhi Linux',
 'Canaima',
 'CentOS',
 'Chakra',
 'Chrome OS',
 'Clear Linux OS',
 'ClearOS',
 'CrunchBang Linux',
 'Damn Small Linux',
 'Debian',
 'Debian Edu',
 'Devuan',
 'Deepin',
 'Dragora GNU/Linux-Libre',
 'dyne:bolic',
 'Elementary OS',
 'ELinOS',
 'Emdebian Grip',
 'EndeavourOS',
 'Fedora',
 'Freespire',
 'Gentoo Linux',
 'Guix System',
 'gNewSense',
 'Grml',
 'Hyperbola GNU/Linux-libre',
 'Kali Linux',
 'Knoppix',
 'Kodibuntu',
 'Korora',
 'LibreCMC',
 'Linspire',
 'Linux Mint',
 'Linux Lite',
 'Mageia',
 'Mandriva Linux',
 'Manjaro Linux',
 'MEPIS',
 'Musix GNU+Linux',
 'Netrunner',
 'NixOS',
 'Novell Open Enterprise Server',
 'OpenELEC',
 'openSUSE',
 'OpenWrt',
 'OpenMandriva Lx',
 'Oracle Linux',
 'Parabola GNU/Linux-libre',
 'Pardus',
 'Parrot OS',
 'Parsix',
 'Parted Magic',
 'PCLinuxOS',
 'Pop! OS',
 'Pentoo',
 'Porteus',
 'Puppy Linux',
 'PureOS',
 'Qubes OS',
 'Red Hat Enterp

In [ ]:
# extract table data (the rest of the table)
td = browser.page.find_all("td")

In [ ]:
td

[<td class="mbox-image"><div style="width:52px"><img alt="" data-file-height="40" data-file-width="40" decoding="async" height="40" src="//upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/40px-Ambox_important.svg.png" srcset="//upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/60px-Ambox_important.svg.png 1.5x, //upload.wikimedia.org/wikipedia/en/thumb/b/b4/Ambox_important.svg/80px-Ambox_important.svg.png 2x" width="40"/></div></td>,
 <td class="mbox-text"><div class="mbox-text-span"><div class="mw-collapsible" style="width:95%; margin: 0.2em 0;"><b>This article has multiple issues.</b> Please help <b><a class="external text" href="https://en.wikipedia.org/w/index.php?title=Comparison_of_Linux_distributions&amp;action=edit">improve it</a></b> or discuss these issues on the <b><a href="/wiki/Talk:Comparison_of_Linux_distributions" title="Talk:Comparison of Linux distributions">talk page</a></b>. <small><i>(<a href="/wiki/Help:Maintenance_template_removal" 

In [ ]:
# tidy up and slice off non-table elements
columns = [value.text.replace("\n", "") for value in td]
columns = columns[6:1051]

In [ ]:
column_names = ["Founder", 
                "Maintainer", 
                "Initial_Release_Year", 
                "Current_Stable_Version", 
                "Security_Updates", 
                "Release_Date", 
                "System_Distribution_Commitment", 
                "Forked_From", 
                "Target_Audience", 
                "Cost", 
                "Status"]

In [ ]:
dictionary = {"Distribution": distribution}

In [ ]:
# insert column names and their data into a dictionary
for idx, key in enumerate(column_names):
    dictionary[key] = columns[idx:][::11]

In [ ]:
# convert dictionary to data frame
df = pd.DataFrame(data = dictionary)

In [ ]:
# create new database and cursor
connection = sqlite3.connect("linux_distro.db")
cursor = connection.cursor()

In [ ]:
# create database table and insert all data frame rows
cursor.execute("create table IF NOT EXISTS linux (Distribution, " + ",".join(column_names)+ ")")
for i in range(len(df)):
    cursor.execute("insert into linux values (?,?,?,?,?,?,?,?,?,?,?,?)", df.iloc[i])

In [ ]:
cursor.execute("select * from linux where Initial_Release_Year=:c", {"c": "2008"})
for row in cursor:
    print(row)

('CrunchBang Linux', 'Philip Newborough', 'Philip Newborough', '2008', '11[13]', '?', '2013-05-06', 'X', 'Debian', 'desktop', 'None', 'Inactive')
('Kodibuntu', 'The XBMC Project', 'Team Kodi', '2008', '?', '?', '?', 'X', 'Debian, Ubuntu', 'multimedia', 'None', 'Inactive')
('Musix GNU+Linux', 'Musix team', 'Musix team', '2008', '3.0.1', '?', '2014-03-13', '100% Free: GNU FSDG[6]', 'Debian', 'multimedia', 'None', 'Inactive')
('SliTaz GNU/Linux', 'Christophe Lincoln', 'dev team', '2008', 'Rolling', 'Rolling', '2017-12-03', 'X', 'Independent', 'portable', 'None', 'Active')
('TurnKey GNU/Linux', 'Alon Swartz, Liraz Siri', 'TurnKey GNU/Linux Team', '2008', '16.0 Release #12[73]', 'Debian packages as per Debian; TurnKey packages for life of current major version (plus backports per request)', '2020-12-22', 'Debian Social Contract and DFSG', 'Debian', 'server based software appliance library aiming to balance security and ease of use', 'None', 'Active')
('Void Linux', 'Juan Romero Pardines', '

In [ ]:
# PERMANENTLY save inserted data in "linux_distro.db"
connection.commit()

In [ ]:
connection.close()